In [1]:
import pandas as pd

In [2]:
data = "data/00--raw/future_price.csv"

y = pd.read_csv(data, index_col=0, parse_dates=True)
y.head()



/tmp/ipykernel_64117/544167729.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272) have mixed types. Speci

,F14,F14_spread,G14,G14_spread,H14,H14_spread,J14,J14_spread,K14,K14_spread,...,Z24,Z24_spread,F25,F25_spread,G25,G25_spread,K25,K25_spread,Q25,Q25_spread
Data,,,,,,,,,,,,,,,,,,,,,
2014-01-02 00:00:00,"112,91","-0,0169","111,63","-0,028","110,13","-0,0411","109,1","-0,0501","109,41","-0,0474",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-03 00:00:00,"113,19","-0,0158","111,8","-0,0279","110,24","-0,0415","109,2","-0,0505","109,48","-0,0481",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-06 00:00:00,"113,6","-0,008","112,2","-0,0203","110,6","-0,0342","109,65","-0,0425","109,65","-0,0425",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-07 00:00:00,"112,94","-0,0158","111,5","-0,0283","109,93","-0,042","108,92","-0,0508","108,8","-0,0519",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-01-08 00:00:00,"112,8","-0,0172","111,59","-0,0277","109,9","-0,0424","108,99","-0,0504","108,96","-0,0506",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from dataclasses import dataclass
from typing import Dict, Iterable, List, Optional, Tuple, Literal

import re
import calendar
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# ===================== Config & Results =====================

@dataclass
class Config:
    """End-to-end configuration for futures processing and dispersion."""
    csv_path: str
    output_dir: Optional[str] = None
    spread_regex: str = r"(?i)\b(spread|sprd|basis|diff|dif|bid-?ask)\b"
    two_digit_year_cutoff: int = 70
    trim_quantiles: Optional[Tuple[float, float]] = (0.01, 0.99)
    dispersion_metric: Optional[Literal["price", "spread"]] = None
    dispersion_normalize: Literal["none", "nearby", "daily_median"] = "none"


@dataclass
class Results:
    """Tidy data, summaries, and plot paths."""
    long_df: pd.DataFrame
    summary_horizon: pd.DataFrame
    summary_by_month: pd.DataFrame
    plot_paths: Dict[str, Optional[str]]
    saved_paths: Dict[str, Optional[str]]


# ===================== Core =====================

_FUTS_CODE_TO_MONTH: Dict[str, int] = {
    "F": 1, "G": 2, "H": 3, "J": 4, "K": 5, "M": 6,
    "N": 7, "Q": 8, "U": 9, "V": 10, "X": 11, "Z": 12,
}


def run(cfg: Config) -> Results:
    """Load, tidy, summarize, plot, and dispersion-analysis."""
    y = _load_and_coerce(cfg.csv_path)

    meta = _build_meta(
        y.columns,
        spread_regex=cfg.spread_regex,
        two_digit_year_cutoff=cfg.two_digit_year_cutoff,
    )

    long_df = _to_long(y, meta)

    metric = cfg.dispersion_metric or _choose_metric(long_df)
    print(f"[run] Using metric='{metric}'")

    horizon = _summarize(long_df[long_df["variable"] == metric], ["horizon_months"])
    by_month = _summarize(long_df[long_df["variable"] == metric], ["target_month_of_year"])

    saved_paths = _save_outputs(
        long_df=long_df,
        horizon=horizon,
        by_month=by_month,
        metric=metric,
        output_dir=cfg.output_dir,
    )

    plot_paths: Dict[str, Optional[str]] = {}
    plot_paths["by_horizon"] = _plot_horizon(horizon, cfg.output_dir, metric)
    plot_paths["by_month"] = _plot_by_month(by_month, cfg.output_dir, metric)

    disp_paths = analyze_dispersion(
        long_df=long_df,
        metric=metric,
        normalize=cfg.dispersion_normalize,
        trim_quantiles=cfg.trim_quantiles,
        output_dir=cfg.output_dir,
    )
    plot_paths.update(disp_paths)

    print("[run] Done.")
    return Results(long_df, horizon, by_month, plot_paths, saved_paths)


# ===================== IO & Parsing =====================

def _load_and_coerce(csv_path: str) -> pd.DataFrame:
    """Load CSV (date index) and coerce numerics with pt-BR handling."""
    p = Path(csv_path)
    if not p.exists():
        raise FileNotFoundError(f"CSV not found at {p}")

    df = pd.read_csv(p, index_col=0, parse_dates=True, low_memory=False).sort_index()

    df = df.apply(_coerce_brazilian_numeric, axis=0)
    print(f"[load] Loaded {df.shape[0]} rows × {df.shape[1]} cols from {p}")
    return df


def _coerce_brazilian_numeric(col: pd.Series) -> pd.Series:
    """Convert strings with decimal comma to floats."""
    if pd.api.types.is_numeric_dtype(col):
        return col.astype(float)

    s = col.astype("string").str.strip()
    s = s.replace({"": np.nan, "nan": np.nan, "None": np.nan})

    both = s.str.contains(r"\d\.\d", na=False) & s.str.contains(",", na=False)
    comma = s.str.contains(",", na=False) & ~both

    s = s.where(~both, s.str.replace(".", "", regex=False))
    s = s.where(~both, s.str.replace(",", ".", regex=False))
    s = s.where(~comma, s.str.replace(",", ".", regex=False))
    s = s.str.replace(" ", "", regex=False)

    return pd.to_numeric(s, errors="coerce")


def _build_meta(
    cols: Iterable[str],
    spread_regex: str,
    two_digit_year_cutoff: int
) -> pd.DataFrame:
    """Detect column delivery month and variable kind."""
    pat_spread = re.compile(spread_regex)
    rows: List[Tuple[str, Optional[pd.Timestamp], str]] = []

    for c in cols:
        tm = _parse_contract_month(c, two_digit_year_cutoff)
        if tm is None:
            continue
        var = "spread" if pat_spread.search(c) else "price"
        rows.append((c, tm, var))

    meta = pd.DataFrame(rows, columns=["orig_col", "target_month", "variable"])
    if meta.empty:
        raise ValueError("No columns with recognizable (month, year) found.")

    print(f"[meta] Contract-mapped columns: {meta.shape[0]}")
    return meta


def _parse_contract_month(col: str, cutoff: int) -> Optional[pd.Timestamp]:
    """Parse delivery month from a column name."""
    s = col.strip()
    slow = s.lower()

    m = re.search(r"(?P<y>\d{4})[-_/\.]?(?P<m>\d{1,2})\b", s)
    if m:
        y4 = int(m.group("y")); mm = int(m.group("m"))
        if 1 <= mm <= 12:
            return pd.Timestamp(y4, mm, 1)

    month_names = (
        "jan|january|feb|february|mar|march|apr|april|may|jun|june|jul|july|"
        "aug|august|sep|sept|september|oct|october|nov|november|dec|december"
    )
    m = re.search(rf"(?P<mon>{month_names})\D{{0,3}}(?P<yr>\d{{2,4}})\b", slow)
    if m:
        mon_map = {
            "jan": 1, "january": 1, "feb": 2, "february": 2,
            "mar": 3, "march": 3, "apr": 4, "april": 4, "may": 5,
            "jun": 6, "june": 6, "jul": 7, "july": 7, "aug": 8, "august": 8,
            "sep": 9, "sept": 9, "september": 9, "oct": 10, "october": 10,
            "nov": 11, "november": 11, "dec": 12, "december": 12,
        }
        mm = mon_map[m.group("mon")]
        yr = int(m.group("yr"))
        if yr < 100:
            yr = 2000 + yr if yr < cutoff else 1900 + yr
        return pd.Timestamp(yr, mm, 1)

    m = re.search(r"\b([FGHJKMNQUVXZ])\s*([0-9]{2,4})\b", s.upper())
    if m:
        code = m.group(1)
        yr = int(m.group(2))
        if yr < 100:
            yr = 2000 + yr if yr < cutoff else 1900 + yr
        mm = _FUTS_CODE_TO_MONTH.get(code)
        if mm:
            return pd.Timestamp(yr, mm, 1)

    return None


# ===================== Reshape & Summaries =====================

def _to_long(y: pd.DataFrame, meta: pd.DataFrame) -> pd.DataFrame:
    """Tidy long: trade_date, target_month, horizon, variable, value."""
    y2 = y.loc[:, meta["orig_col"]].copy()
    y2.index.name = "trade_date"

    y2.columns = pd.MultiIndex.from_frame(meta[["target_month", "variable"]])
    y2.columns.set_names(["target_month", "variable"], inplace=True)

    long = y2.stack(["target_month", "variable"]).rename("value").reset_index()

    td = pd.to_datetime(long["trade_date"]).dt.normalize()
    tm = pd.to_datetime(long["target_month"]).dt.normalize()

    long["horizon_days"] = (tm - td).dt.days
    long["horizon_months"] = (tm.dt.year - td.dt.year) * 12 + (tm.dt.month - td.dt.month)
    long["target_month_of_year"] = tm.dt.month

    long = long.loc[long["horizon_days"] >= 0].reset_index(drop=True)

    cols = [
        "trade_date", "target_month", "target_month_of_year",
        "horizon_days", "horizon_months", "variable", "value",
    ]
    long = long[cols]
    print(f"[long] Tidy rows: {long.shape[0]}")
    return long


def _choose_metric(long_df: pd.DataFrame) -> str:
    """Pick 'spread' if present with data; else 'price'."""
    has_spread = (long_df["variable"] == "spread").any() and long_df.loc[long_df["variable"] == "spread", "value"].notna().any()
    return "spread" if has_spread else "price"


def _summarize(df: pd.DataFrame, by: List[str]) -> pd.DataFrame:
    """Mean, std, count, q25, q75 grouped by given keys."""
    if df.empty:
        return pd.DataFrame(columns=by + ["mean", "std", "count", "q25", "q75"])

    g = df.groupby(by, observed=True)["value"]
    out = g.agg(
        mean="mean",
        std="std",
        count="count",
        q25=lambda x: x.quantile(0.25),
        q75=lambda x: x.quantile(0.75),
    ).reset_index()
    return out


# ===================== Output =====================

def _ensure_outdir(output_dir: Optional[str]) -> Optional[Path]:
    if output_dir is None:
        return None
    p = Path(output_dir)
    p.mkdir(parents=True, exist_ok=True)
    return p


def _save_outputs(
    long_df: pd.DataFrame,
    horizon: pd.DataFrame,
    by_month: pd.DataFrame,
    metric: str,
    output_dir: Optional[str],
) -> Dict[str, Optional[str]]:
    """Save CSVs. File names reflect the chosen metric."""
    outdir = _ensure_outdir(output_dir)
    paths: Dict[str, Optional[str]] = {"long_csv": None, "summary_horizon": None, "summary_by_month": None}
    if outdir is None:
        return paths

    paths["long_csv"] = str(outdir / "futures_long_tidy.csv")
    paths["summary_horizon"] = str(outdir / f"summary_horizon_{metric}.csv")
    paths["summary_by_month"] = str(outdir / f"summary_month_{metric}.csv")

    long_df.to_csv(paths["long_csv"], index=False)
    horizon.to_csv(paths["summary_horizon"], index=False)
    by_month.to_csv(paths["summary_by_month"], index=False)

    print(f"[save] CSVs saved under {outdir}")
    return paths


def _plot_horizon(horizon: pd.DataFrame, output_dir: Optional[str], metric: str) -> Optional[str]:
    """Errorbar mean±std vs horizon months."""
    if horizon.empty:
        print("[plot_horizon] No data to plot.")
        return None

    hs = horizon.dropna(subset=["mean"]).sort_values("horizon_months")
    if hs.empty:
        print("[plot_horizon] No non-NaN means to plot.")
        return None

    plt.figure()
    plt.errorbar(hs["horizon_months"], hs["mean"], yerr=hs["std"], fmt="-o")
    plt.xlabel("Horizon (months)")
    plt.ylabel(f"{metric.capitalize()} (mean ± std)")
    plt.title(f"{metric.capitalize()} by horizon")
    plt.grid(True)

    outdir = _ensure_outdir(output_dir)
    path = None
    if outdir:
        path = str(outdir / f"plot_{metric}_by_horizon.png")
        plt.tight_layout()
        plt.savefig(path, dpi=160)
        print(f"[plot_horizon] Saved to {path}")

    plt.close()
    return path


def _plot_by_month(by_month: pd.DataFrame, output_dir: Optional[str], metric: str) -> Optional[str]:
    """Errorbar mean±std vs delivery month with aliases on x-axis."""
    if by_month.empty:
        print("[plot_by_month] No data to plot.")
        return None

    md = by_month.dropna(subset=["mean"]).sort_values("target_month_of_year")
    if md.empty:
        print("[plot_by_month] No non-NaN means to plot.")
        return None

    month_labels = [calendar.month_abbr[i].upper() for i in range(1, 13)]

    plt.figure()
    plt.errorbar(md["target_month_of_year"], md["mean"], yerr=md["std"], fmt="-o")
    plt.xticks(ticks=np.arange(1, 13, 1), labels=month_labels)
    plt.xlabel("Delivery month")
    plt.ylabel(f"{metric.capitalize()} (mean ± std)")
    plt.title(f"{metric.capitalize()} by delivery month")
    plt.grid(True)

    outdir = _ensure_outdir(output_dir)
    path = None
    if outdir:
        path = str(outdir / f"plot_{metric}_by_month.png")
        plt.tight_layout()
        plt.savefig(path, dpi=160)
        print(f"[plot_by_month] Saved to {path}")

    plt.close()
    return path


# ===================== Dispersion (scale-free) =====================

def analyze_dispersion(
    long_df: pd.DataFrame,
    metric: Literal["price", "spread"] = "price",
    normalize: Literal["none", "nearby", "daily_median"] = "none",
    trim_quantiles: Optional[Tuple[float, float]] = (0.01, 0.99),
    output_dir: Optional[str] = None,
) -> Dict[str, Optional[str]]:
    """Compute CV, robust CV, IQR ribbon and plot them."""
    df = long_df.loc[long_df["variable"] == metric, ["trade_date", "horizon_months", "value"]].copy()
    if df.empty:
        print(f"[analyze_dispersion] No rows for metric='{metric}'.")
        return {"summary_csv": None, "cv_plot": None, "robust_cv_plot": None, "iqr_plot": None}

    df["trade_date"] = pd.to_datetime(df["trade_date"]).dt.normalize()

    if normalize == "nearby":
        base = (
            df.loc[df.groupby("trade_date")["horizon_months"].transform("min") == df["horizon_months"],
                   ["trade_date", "value"]]
            .rename(columns={"value": "base_value"})
        )
        df = df.merge(base, on="trade_date", how="left")
        df["value"] = df["value"] / df["base_value"]
        print("[analyze_dispersion] Normalized by daily nearby (ratio).")

    elif normalize == "daily_median":
        med = df.groupby("trade_date")["value"].transform("median")
        df["value"] = df["value"] / med
        print("[analyze_dispersion] Normalized by daily median (ratio).")

    if trim_quantiles is not None:
        ql, qh = trim_quantiles
        trims = df.groupby("horizon_months")["value"].quantile([ql, qh]).unstack()
        df = df.join(trims, on="horizon_months", rsuffix="_trim")
        df["value"] = df["value"].clip(lower=df[ql], upper=df[qh])
        df.drop(columns=[ql, qh], inplace=True)
        print(f"[analyze_dispersion] Winsorized per horizon at {trim_quantiles}.")

    summary = _dispersion_summary(df)

    return _save_and_plot_dispersion(summary, metric, normalize, output_dir)


def _dispersion_summary(df: pd.DataFrame) -> pd.DataFrame:
    """Mean, std, CV, robust CV, median, IQR by horizon."""
    g = df.groupby("horizon_months")["value"]

    median = g.median()
    q25 = g.quantile(0.25)
    q75 = g.quantile(0.75)
    iqr = q75 - q25

    mad = g.apply(lambda x: (x - x.median()).abs().median())
    madn = 1.4826 * mad

    mean = g.mean()
    std = g.std(ddof=1)

    out = pd.DataFrame({
        "horizon_months": mean.index,
        "mean": mean.values,
        "std": std.values,
        "cv": (std / mean.replace(0, np.nan)).values,
        "median": median.values,
        "madn": madn.values,
        "rcv": (madn / median.replace(0, np.nan)).values,
        "q25": q25.values,
        "q75": q75.values,
        "iqr": iqr.values,
        "iqr_over_median": (iqr / median.replace(0, np.nan)).values,
        "count": g.size().values,
    }).sort_values("horizon_months").reset_index(drop=True)

    return out


def _save_and_plot_dispersion(
    summary: pd.DataFrame,
    metric: str,
    normalize: str,
    output_dir: Optional[str],
) -> Dict[str, Optional[str]]:
    """Emit CSV and dispersion plots."""
    pdir = _ensure_outdir(output_dir)
    tag = f"{metric}_{normalize}"

    csv_path = str(pdir / f"dispersion_{tag}.csv") if pdir else None
    if csv_path:
        summary.to_csv(csv_path, index=False)
        print(f"[dispersion] Summary -> {csv_path}")

    cv_path = _plot_cv(summary, f"{metric.capitalize()} CV by horizon ({normalize})", "cv",
                       str(pdir / f"plot_cv_{tag}.png") if pdir else None)

    rcv_path = _plot_cv(summary, f"{metric.capitalize()} robust CV (MADn/median) by horizon ({normalize})", "rcv",
                        str(pdir / f"plot_rcv_{tag}.png") if pdir else None)

    iqr_path = _plot_median_iqr(summary, metric, normalize,
                                str(pdir / f"plot_median_iqr_{tag}.png") if pdir else None)

    return {"summary_csv": csv_path, "cv_plot": cv_path, "robust_cv_plot": rcv_path, "iqr_plot": iqr_path}


def _plot_cv(summary: pd.DataFrame, title: str, y_col: str, output_path: Optional[str]) -> Optional[str]:
    """Plot CV-like measure vs horizon."""
    if summary.empty or summary[y_col].isna().all():
        print(f"[plot_cv] Nothing to plot for {y_col}.")
        return None

    s = summary.dropna(subset=[y_col]).sort_values("horizon_months")

    plt.figure()
    plt.plot(s["horizon_months"], s[y_col], marker="o")
    plt.xlabel("Horizon (months)")
    plt.ylabel(y_col.upper())
    plt.title(title)
    plt.grid(True)

    if output_path:
        plt.tight_layout()
        plt.savefig(output_path, dpi=160)
        print(f"[plot_cv] Saved -> {output_path}")

    plt.close()
    return output_path


def _plot_median_iqr(summary: pd.DataFrame, metric: str, normalize: str, output_path: Optional[str]) -> Optional[str]:
    """Median with IQR ribbon vs horizon."""
    if summary.empty:
        print("[plot_median_iqr] Nothing to plot.")
        return None

    s = summary.sort_values("horizon_months")

    plt.figure()
    plt.plot(s["horizon_months"], s["median"], marker="o")
    plt.fill_between(s["horizon_months"], s["q25"], s["q75"], alpha=0.25, label="IQR")
    plt.xlabel("Horizon (months)")
    plt.ylabel(f"{metric.capitalize()} (median & IQR)")
    plt.title(f"{metric.capitalize()} median with IQR by horizon ({normalize})")
    plt.grid(True)
    plt.legend()

    if output_path:
        plt.tight_layout()
        plt.savefig(output_path, dpi=160)
        print(f"[plot_median_iqr] Saved -> {output_path}")

    plt.close()
    return output_path


# ===================== Example =====================

if __name__ == "__main__":
    cfg = Config(
        csv_path="data/00--raw/future_price.csv",
        output_dir="data/10--derived/futures-analysis",
        dispersion_normalize="nearby"  # try "none" | "nearby" | "daily_median"
    )
    _ = run(cfg)


[load] Loaded 2753 rows × 272 cols from data/00--raw/future_price.csv
[meta] Contract-mapped columns: 136
[long] Tidy rows: 23477
[run] Using metric='price'
[save] CSVs saved under data/10--derived/futures-analysis


/tmp/ipykernel_64117/7828156.py:201: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  long = y2.stack(["target_month", "variable"]).rename("value").reset_index()


[plot_horizon] Saved to data/10--derived/futures-analysis/plot_price_by_horizon.png
[plot_by_month] Saved to data/10--derived/futures-analysis/plot_price_by_month.png
[analyze_dispersion] Normalized by daily nearby (ratio).
[analyze_dispersion] Winsorized per horizon at (0.01, 0.99).
[dispersion] Summary -> data/10--derived/futures-analysis/dispersion_price_nearby.csv
[plot_cv] Saved -> data/10--derived/futures-analysis/plot_cv_price_nearby.png
[plot_cv] Saved -> data/10--derived/futures-analysis/plot_rcv_price_nearby.png
[plot_median_iqr] Saved -> data/10--derived/futures-analysis/plot_median_iqr_price_nearby.png
[run] Done.


In [4]:
# -*- coding: utf-8 -*-
"""
Prediction-aware alignment for Boi Gordo futures vs. actuals (B3 & Cepea).

Inputs
------
ACTUALS_CSV = "data/00--raw/price.csv"
  Columns (pt-BR format, example):
    Data, Boi gordo B3, Boi gordo Cepea, Spread Absoluto, Spread Relativo
  Dates like "01.09.2020"; numbers like "242,4".

PREDICTIONS_CSV = "data/00--raw/future-price.csv"
  Wide panel with one row per issue date and many contract columns:
    Data, F14, F14_spread, G14, G14_spread, H14, ... , Z25, Z25_spread
  Dates like "2014-01-02 00:00:00"; numbers like "112,91".

Behavior
--------
For each issue date d and horizon h ∈ {3,6,12} months:
- target_date = d + h months
- compute the contract code for (target_date.year, target_date.month), e.g. Mar/2014 -> "H14"
- y_pred = futures[d, code]
- y_actual = actuals[target_date] (for B3 and Cepea; daily ffilled)

Outputs (written to OUT_DIR = "data/10--derived/prediction_aware")
------------------------------------------------------------------
- plot_date_aligned_3m.png
- plot_date_aligned_6m.png
- plot_date_aligned_12m.png
- plot_abs_error_violins.png
- prediction_error_summary.csv

Notes
-----
- Locale-aware numeric parsing (pt-BR).
- Actuals are resampled to daily and forward-filled.
- Uses matplotlib only (no seaborn), one chart per figure, no explicit colors.
"""

import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import AutoDateLocator, ConciseDateFormatter

# ----------------------
# Config (edit as needed)
# ----------------------
ACTUALS_CSV = "data/00--raw/price.csv"
PREDICTIONS_CSV = "data/00--raw/future_price.csv"
OUT_DIR = "data/10--derived/prediction-aware"
HORIZONS = [3, 6, 12]  # months

# Month code map used by futures:
# F=Jan(1), G=Feb(2), H=Mar(3), J=Apr(4), K=May(5), M=Jun(6),
# N=Jul(7), Q=Aug(8), U=Sep(9), V=Oct(10), X=Nov(11), Z=Dec(12)
MONTH_TO_CODE = {1: 'F', 2: 'G', 3: 'H', 4: 'J', 5: 'K', 6: 'M',
                 7: 'N', 8: 'Q', 9: 'U', 10: 'V', 11: 'X', 12: 'Z'}
CODE_RE = re.compile(r'^[FGHJKMNQUVXZ]\d{2}$')  # e.g., F14, H20, Z25


# ---------
# Utilities
# ---------
def _coerce_numeric_ptbr(s: pd.Series) -> pd.Series:
    """Parse strings like '2.345,67' -> 2345.67. Leaves numeric dtypes unchanged."""
    if s.dtype.kind in "iufc":
        return s
    s2 = s.astype(str).str.strip()
    has_comma = s2.str.contains(",", regex=False, na=False)
    s2a = s2.where(~has_comma,
                   s2.str.replace(".", "", regex=False).str.replace(",", ".", regex=False))
    return pd.to_numeric(s2a, errors="coerce")


def _ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


# -------------------
# Load & prep actuals
# -------------------
def _read_actuals(path: str):
    df = pd.read_csv(path)
    # Date column is "Data" with dd.mm.yyyy pattern (pt-BR)
    date_col = None
    if 'Data' in df.columns:
        date_col = 'Data'
    else:
        # fallback scan
        for c in df.columns:
            try:
                parsed = pd.to_datetime(df[c], errors='coerce', dayfirst=True)
                if parsed.notna().mean() > 0.6:
                    date_col = c
                    break
            except Exception:
                pass
    if date_col is None:
        raise ValueError("Could not find a date column in actuals (expected 'Data').")

    df[date_col] = pd.to_datetime(df[date_col], errors='coerce', dayfirst=True)
    df = df.dropna(subset=[date_col]).sort_values(date_col).reset_index(drop=True)

    # Identify B3 / Cepea columns by name
    def _find_col(name_part: str):
        for c in df.columns:
            if name_part.lower() in c.lower():
                return c
        return None

    b3_col = _find_col("Boi gordo B3")
    cepea_col = _find_col("Boi gordo Cepea")
    if b3_col is None or cepea_col is None:
        raise ValueError("Could not find 'Boi gordo B3' and/or 'Boi gordo Cepea' in actuals.")

    # Coerce numeric (pt-BR to float)
    df[b3_col] = _coerce_numeric_ptbr(df[b3_col])
    df[cepea_col] = _coerce_numeric_ptbr(df[cepea_col])

    # Build daily forward-filled index
    adf = df[[date_col, b3_col, cepea_col]].copy().set_index(date_col).sort_index()
    full_idx = pd.date_range(adf.index.min(), adf.index.max(), freq='D')
    adf = adf.reindex(full_idx).ffill()
    adf.index.name = 'date'
    return adf, b3_col, cepea_col


# -----------------------
# Load & prep futures wide
# -----------------------
def _read_futures_wide(path: str):
    fut = pd.read_csv(path)
    # Date column is "Data" with ISO + time (e.g., 2014-01-02 00:00:00)
    if 'Data' not in fut.columns:
        raise ValueError("Futures file is missing 'Data' column.")
    fut['Data'] = pd.to_datetime(fut['Data'], errors='coerce', dayfirst=False)
    fut = fut.dropna(subset=['Data']).sort_values('Data').reset_index(drop=True)

    # Identify pure contract columns like 'F14', 'H21', etc. (ignore *_spread)
    contract_cols = [c for c in fut.columns if CODE_RE.match(c)]
    # Coerce numeric pt-BR for those columns
    for c in contract_cols:
        fut[c] = _coerce_numeric_ptbr(fut[c])

    return fut, contract_cols


def _code_for_ym(year: int, month: int) -> str:
    """Return futures code like 'H14' for Mar/2014."""
    code = MONTH_TO_CODE[month]
    yy = year % 100
    return f"{code}{yy:02d}"


def _build_predictions_from_futures(fut: pd.DataFrame, contract_cols: list, horizons=(3, 6, 12)):
    """
    Convert wide futures panel into a long predictions table with:
      issue_date, target_date, horizon_m, y_pred
    For each row (issue date) and each horizon h, pick the column whose code equals (issue_date + h months).
    """
    fut = fut.copy()
    issue_dates = fut['Data']
    out_frames = []
    colset = set(contract_cols)

    for h in horizons:
        # Compute target dates = issue_date + h months (month-preserving)
        target_dates = issue_dates + pd.DateOffset(months=h)
        # Build the code string for each row's target Y-M
        codes = [_code_for_ym(d.year, d.month) for d in target_dates]
        # Extract predicted values row-by-row from the matching column (if present)
        # We'll index by integer position to avoid alignment surprises.
        preds = []
        for i, code in enumerate(codes):
            if code in colset:
                preds.append(fut.at[i, code])
            else:
                preds.append(np.nan)
        df_h = pd.DataFrame({
            'issue_date': issue_dates.values,
            'target_date': target_dates.values,
            'horizon_m': h,
            'y_pred': preds
        })
        out_frames.append(df_h)

    preds_long = pd.concat(out_frames, ignore_index=True)
    preds_long = preds_long.dropna(subset=['y_pred']).reset_index(drop=True)
    return preds_long


# -------------------------
# Merge with actuals & plots
# -------------------------
def _attach_actuals(preds: pd.DataFrame, actuals: pd.DataFrame, b3_col: str, cepea_col: str):
    out = preds.copy()
    out['actual_B3'] = actuals.reindex(out['target_date'])[b3_col].values
    out['actual_Cepea'] = actuals.reindex(out['target_date'])[cepea_col].values
    out = out.dropna(subset=['actual_B3', 'actual_Cepea']).reset_index(drop=True)
    out['x_date'] = out['issue_date']  # align on issue date for plotting
    return out


def _plot_date_aligned(df_h: pd.DataFrame, horizon: int, outpath: str):
    plt.figure(figsize=(10, 5))
    plt.plot(df_h['x_date'], df_h['y_pred'], label='Prediction')
    plt.plot(df_h['x_date'], df_h['actual_B3'], label='Actual B3 (shifted)')
    plt.plot(df_h['x_date'], df_h['actual_Cepea'], label='Actual Cepea (shifted)')
    plt.title(f'Prediction-aware alignment: {horizon}m horizon')
    plt.xlabel('Issue date')
    plt.ylabel('Price')
    plt.grid(True, alpha=0.3)
    plt.legend()
    ax = plt.gca()
    locator = AutoDateLocator()
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(ConciseDateFormatter(locator))
    plt.tight_layout()
    plt.savefig(outpath, dpi=160)
    plt.close()


def _plot_violin_abs_errors(err_long: pd.DataFrame, outpath: str):
    horizons = sorted(err_long['horizon_m'].unique())
    positions, datasets, medians, xticks, xticklabels = [], [], [], [], []
    for i, h in enumerate(horizons):
        left = i*3 + 1.0 - 0.3
        right = i*3 + 1.0 + 0.3
        b3 = err_long[(err_long['horizon_m']==h) & (err_long['series']=='B3')]['abs_error'].dropna().values
        cp = err_long[(err_long['horizon_m']==h) & (err_long['series']=='Cepea')]['abs_error'].dropna().values
        positions.extend([left, right])
        datasets.extend([b3, cp])
        medians.extend([np.median(b3) if len(b3)>0 else np.nan,
                        np.median(cp) if len(cp)>0 else np.nan])
        xticks.append(i*3 + 1.0)
        xticklabels.append(f'{h}m')

    plt.figure(figsize=(10, 5))
    plt.violinplot(datasets, positions=positions, showmeans=False, showmedians=False, showextrema=False)
    plt.scatter(positions, medians, marker='o', zorder=3, label='Median (dot)')
    plt.xticks(xticks, xticklabels)
    plt.xlabel('Horizon')
    plt.ylabel('|Error|')
    plt.title('Absolute error by horizon (B3 vs Cepea)')
    plt.grid(True, alpha=0.3)
    plt.text(0.02, 0.95, 'Left = B3, Right = Cepea', transform=plt.gca().transAxes, va='top')
    plt.tight_layout()
    plt.savefig(outpath, dpi=160)
    plt.close()


def _summarize_errors(df: pd.DataFrame):
    rows = []
    for s in ['B3', 'Cepea']:
        a = df['actual_'+s]
        e = a - df['y_pred']
        ae = e.abs()
        mape = (ae / np.where(a != 0, a, np.nan)).mean()
        rows.append({
            'series': s,
            'count': int(ae.notna().sum()),
            'mae': float(ae.mean()),
            'median_ae': float(ae.median()),
            'rmse': float(np.sqrt((e**2).mean())),
            'bias': float(e.mean()),
            'mape': float(mape) if not np.isnan(mape) else np.nan,
        })
    return pd.DataFrame(rows)


# --------------
# Main pipeline
# --------------
def run_pipeline():
    _ensure_dir(OUT_DIR)

    print("Reading actuals:", ACTUALS_CSV)
    actuals, b3_col, cepea_col = _read_actuals(ACTUALS_CSV)
    print("Detected actual columns -> B3:", b3_col, "| Cepea:", cepea_col)

    print("Reading futures:", PREDICTIONS_CSV)
    fut, contract_cols = _read_futures_wide(PREDICTIONS_CSV)
    print(f"Detected {len(contract_cols)} contract columns (e.g., first 10):", contract_cols[:10])

    preds = _build_predictions_from_futures(fut, contract_cols, horizons=HORIZONS)
    if preds.empty:
        print("No predictions could be constructed for horizons:", HORIZONS)
        return {"note": "No predictions for requested horizons."}

    merged = _attach_actuals(preds, actuals, b3_col, cepea_col)
    if merged.empty:
        print("No rows after attaching actuals (date overlap may be empty).")
        return {"note": "No overlap between predictions target dates and actuals."}

    outputs = {}

    # Date-aligned plots
    for h in HORIZONS:
        df_h = merged[merged['horizon_m'] == h].copy()
        if df_h.empty:
            continue
        outpath = os.path.join(OUT_DIR, f"plot_date_aligned_{h}m.png")
        _plot_date_aligned(df_h, h, outpath)
        outputs[f"{h}m_plot"] = outpath
        print("Wrote", outpath)

    # Violin of |error|
    err_rows = []
    for _, r in merged.iterrows():
        for s in ['B3', 'Cepea']:
            e = r[f'actual_{s}'] - r['y_pred']
            err_rows.append({'horizon_m': int(r['horizon_m']), 'series': s, 'abs_error': abs(e)})
    err_long = pd.DataFrame(err_rows)

    violin_path = os.path.join(OUT_DIR, "plot_abs_error_violins.png")
    _plot_violin_abs_errors(err_long, violin_path)
    outputs["violins"] = violin_path
    print("Wrote", violin_path)

    # Summary CSV
    summary_rows = []
    for h in HORIZONS:
        df_h = merged[merged['horizon_m'] == h].copy()
        if df_h.empty:
            continue
        s_df = _summarize_errors(df_h)
        s_df.insert(0, 'horizon_m', h)
        summary_rows.append(s_df)
    if summary_rows:
        summary = pd.concat(summary_rows, ignore_index=True)
        summary_path = os.path.join(OUT_DIR, "prediction_error_summary.csv")
        summary.to_csv(summary_path, index=False)
        outputs["summary_csv"] = summary_path
        print("Wrote", summary_path)

    return outputs


if __name__ == "__main__":
    out = run_pipeline()
    print("Outputs:", out)


Reading actuals: data/00--raw/price.csv
Detected actual columns -> B3: Boi gordo B3 | Cepea: Boi gordo Cepea
Reading futures: data/00--raw/future_price.csv


/tmp/ipykernel_64117/2861966300.py:134: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272) have mixed types. Sp

Detected 136 contract columns (e.g., first 10): ['F14', 'G14', 'H14', 'J14', 'K14', 'M14', 'N14', 'Q14', 'U14', 'V14']
Wrote data/10--derived/prediction-aware/plot_date_aligned_3m.png
Wrote data/10--derived/prediction-aware/plot_date_aligned_6m.png
Wrote data/10--derived/prediction-aware/plot_date_aligned_12m.png
Wrote data/10--derived/prediction-aware/plot_abs_error_violins.png
Wrote data/10--derived/prediction-aware/prediction_error_summary.csv
Outputs: {'3m_plot': 'data/10--derived/prediction-aware/plot_date_aligned_3m.png', '6m_plot': 'data/10--derived/prediction-aware/plot_date_aligned_6m.png', '12m_plot': 'data/10--derived/prediction-aware/plot_date_aligned_12m.png', 'violins': 'data/10--derived/prediction-aware/plot_abs_error_violins.png', 'summary_csv': 'data/10--derived/prediction-aware/prediction_error_summary.csv'}
